# Traditional Sentiment Analysis
### Tf-idf with logistic regression

from https://github.com/tthustla/yet_another_tiwtter_sentiment_analysis_part1/blob/master/Yet_Another_Twitter_Sentiment_Analysis_part1-Copy1.ipynb

also check https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4

and https://sajalsharma.com/portfolio/sentiment_analysis_tweets to implement lexicon with ML algorithm

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import json
# .txt file with text already pre-processed
# sentiment<SPACE>tweet per line

# import training and test data
training_file = '../scripts/train_binary.json'
valid_file = '../scripts/valid_binary.json'
test_file = '../scripts/test_binary.json'

with open(training_file, 'r') as json_file:
    training = json.load(json_file)
        
with open(valid_file, 'r') as json_file:
    valid = json.load(json_file)
        
with open(test_file, 'r') as json_file:
    test = json.load(json_file)

training_data = training['data']
training_texts = [tweet['cleaned_text'] for tweet in training_data]
training_labels = [tweet['sentiment'] for tweet in training_data]

test_data = test['data']
test_texts = [tweet['cleaned_text'] for tweet in test_data]
test_labels = [tweet['sentiment'] for tweet in test_data]

valid_data = valid['data']
valid_texts = [tweet['cleaned_text'] for tweet in valid_data]
valid_labels = [tweet['sentiment'] for tweet in valid_data]

train_data = np.concatenate((training_data, valid_data), axis=None)
train_texts = np.concatenate((training_texts, valid_texts), axis=None)
train_labels = np.concatenate((training_labels, valid_labels), axis=None)
# generate dataframes
#columns = ['sentiment', 'color']
#df = pd.DataFrame(dataset)

In [ ]:
np.unique(train_labels)

### Import lexicon

In [ ]:
with open('../../data/senticon-en.json', 'r') as json_file:
      lexicon = json.load(json_file)
lexicon_pos = lexicon['positive']
lexicon_neg = lexicon['negative']

## BoW like in https://sajalsharma.com/portfolio/sentiment_analysis_tweets

In [ ]:
# english stopwords would be removed
from nltk.corpus import stopwords

stopwords = set(stopwords.words('english'))

#generate Bag of Words
def generateBoW(data, ignoreStopwords):
    bow = {}
    for tweet in data:
        for token in tweet['tokens']:
            word = token['token']
            if ignoreStopwords:
                if word not in stopwords:
                    bow[word] = bow.get(word,0) + 1
            else:
                bow[word] = bow.get(word,0) + 1
    return bow

bow_train = generateBoW(train_data, True)
len(bow_train)

In [ ]:
# generate ngrams
from nltk.util import ngrams

# compute up to 3-grams
def extract_ngrams_from_dataset(data, ignoreStopwords):
    ngrams_count = {n:dict() for n in range(1,4)}
    for tweet in data:
        if ignoreStopwords:
            tokens = [el['token'] for el in tweet['tokens'] if el['token'] not in stopwords]
        else:
            tokens = [el['token'] for el in tweet['tokens']]
        for n in range(1,4):
            ngram = ngrams(tokens, n)
            # now you have an n-gram you can do what ever you want
            # yield ngram
            # you can count them for your language model?
            for item in list(ngram):
                if len(list(item)) == 1:
                    item = item[0]
                elif len(list(item)) == 2:
                    item = f'{item[0]}-{item[1]}'
                elif len(list(item)) == 3:
                    item = f'{item[0]}-{item[1]}-{item[2]}'
                ngrams_count[n][item] = ngrams_count[n].get(item, 0) + 1
    return ngrams_count
            
ngrams_list = extract_ngrams_from_dataset(train_data, True)
all_ngrams = {**ngrams_list[1], **ngrams_list[2], **ngrams_list[3]}
len(all_ngrams)

### Generate features

In [ ]:
def extract_ngrams_from_tokens(tokens):
    ngrams_count = {n:dict() for n in range(1,4)}
    for n in range(1,4):
        ngram = ngrams(tokens, n)
        for item in list(ngram):
            if len(list(item)) == 1:
                item = item[0]
            elif len(list(item)) == 2:
                item = f'{item[0]}-{item[1]}'
            elif len(list(item)) == 3:
                item = f'{item[0]}-{item[1]}-{item[2]}'
            ngrams_count[n][item] = ngrams_count[n].get(item, 0) + 1
    return ngrams_count

# convert raw tweets to feature dictionaries
def generateFeatureDicts(data, removeStopwords):
    feature_dicts = []
    for tweet in data:
        # build feature dictionary for tweet, list of ngrams with count to 0
        tweet_feature_dict = all_ngrams.fromkeys(all_ngrams, 0)
        # get tokens
        if removeStopwords:
            tokens = [el['token'] for el in tweet['tokens'] if el['token'] not in stopwords]
        else:
            tokens = [el['token'] for el in tweet['tokens']]
        
        # get ngrams of tweet
        tweet_ngrams_dicts = extract_ngrams_from_tokens(tokens)
        # join the three dictionaries of ngrams
        tweet_ngrams = {**tweet_ngrams_dicts[1], **tweet_ngrams_dicts[2], **tweet_ngrams_dicts[3]}
        
        for gram, count in tweet_ngrams.items():
            tweet_feature_dict[gram] = count
        
        feature_dicts.append(tweet_feature_dict)
        print('done!')
        '''
        for token in tweet['tokens']:
            word = token['token']
            if removeStopwords:
                if word not in stopwords:
                    tweet_feature_dict[word] = tweet_feature_dict.get(word,0) + 1
            else:
                tweet_feature_dict[word] = tweet_feature_dict.get(word,0) + 1
        feature_dicts.append(tweet_feature_dict)
        '''
    return feature_dicts

# syntactical features
'''
the number of occurrences of nouns, adjectives and adverbs are added to the feature vector.
the number of emphasis tokens present in the tweet is added to the feature space
'''
def get_syntactic_features(data):
    syntatic_features_dicts = []
    for tweet in data:
        syntatic_dict = {}
        syntatic_dict['nouns'] = 0
        syntatic_dict['adjectives'] = 0
        syntatic_dict['adverbs'] = 0
        syntatic_dict['emphasis'] = 0
        for token in tweet['tokens']:
            if token['tag'] == 'NOUN':
                syntatic_dict['nouns'] = syntatic_dict['nouns'] + 1
            elif token['tag'] == 'ADJECTIVE':
                syntatic_dict['adjectives'] = syntatic_dict['adjectives'] + 1
            elif token['tag'] == 'ADVERB':
                syntatic_dict['adverbs'] = syntatic_dict['adverbs'] + 1
            elif token['tag'] == 'EMPHASIS':
                syntatic_dict['emphasis'] = syntatic_dict['emphasis'] + 1
        syntatic_features_dicts.append(syntatic_dict)
    
    return syntatic_features_dicts

# lexicon features
def get_lexicon_features(data):
    lexicon_feature_dict = []
    for tweet in data:
        tweet_dict = {}
        pos_pol = []
        neg_pol = []
        for token in tweet['tokens']:
            word = token['token']
            if word in lexicon_pos:
                pos_pol.append(lexicon_pos[word])
            elif word in lexicon_neg:
                neg_pol.append(lexicon_neg[word])
            
            p = len(pos_pol)
            n = len(neg_pol)
            pos_pol = [float(el) for el in pos_pol]
            neg_pol = [float(el) for el in neg_pol]
            tweet_dict['avg-pos'] = np.round(np.mean(pos_pol),3) if p > 0 else 0 
            tweet_dict['avg-neg'] = np.round(np.mean(neg_pol),3) if n > 0 else 0 
            tweet_dict['last-pos'] = pos_pol[-1] if p > 0 else 0
            tweet_dict['last-neg'] = neg_pol[-1] if n > 0 else 0
            tweet_dict['max-pos'] = max(pos_pol) if p > 0 else 0
            tweet_dict['max-neg'] = min(neg_pol) if n > 0 else 0
            # TWEET POLARITY COMPUTATION
            # firs condition: P > N
            if p > n:
                if n > 0:
                    tweet_dict['polarity'] = np.round(1 - n/p,3)
                else:
                    tweet_dict['polarity'] = np.mean(pos_pol)
            elif p < n:
                if p > 0:
                    tweet_dict['polarity'] = np.round(p/n - 1,3)
                else:
                    tweet_dict['polarity'] = np.mean(neg_pol)
            elif p == n:
                tweet_dict['polarity'] = 0
        lexicon_feature_dict.append(tweet_dict)
    
    return lexicon_feature_dict
            
            
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn

from nltk.corpus import opinion_lexicon
positive_words = opinion_lexicon.positive()
negative_words = opinion_lexicon.negative()

In [ ]:
lexicon_features = get_lexicon_features(train_data)

In [ ]:
lexicon_features[:10]

In [ ]:
# generate syntactic feature dictionaries
train_synt_dict = get_syntactic_features(train_data)
test_synt_dict = get_syntactic_features(test_data)
# convert features dictionary to a sparse representation, so that they can be used by sklearn ML algorithms
# https://scikit-learn.org/stable/modules/feature_extraction.html
from sklearn.feature_extraction import DictVectorizer

vectorizer = DictVectorizer()

# Convert feature dictionaries to sparse representations
train_synt_features = vectorizer.fit_transform(train_synt_dict)
test_synt_features = vectorizer.transform(test_synt_dict)
#train_custom_features.toarray()

In [ ]:
# generate ngrams feature dictionaries
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = CountVectorizer(stop_words=stopwords, max_features=100000, ngram_range=(1,3))
#vectorizer = TfidfVectorizer(stop_words=stopwords)

# Generate feature vectors
train_ngram_features = vectorizer.fit_transform(train_texts)
test_ngram_features = vectorizer.transform(test_texts)

In [ ]:
train_lex_dict = get_lexicon_features(train_data)
test_lex_dict = get_lexicon_features(test_data)

vectorizer = DictVectorizer()

# Convert feature dictionaries to sparse representations
train_lex_features = vectorizer.fit_transform(train_lex_dict)
test_lex_features = vectorizer.transform(test_lex_dict)

In [ ]:
from scipy.sparse import hstack, vstack
training_combined = hstack([train_ngram_features, train_synt_features, train_lex_features])
test_combined = hstack([test_ngram_features, test_synt_features, test_lex_features])

In [ ]:
training_combined

### Classification

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
#train_scaler = StandardScaler(with_mean=False)
#scaled_features = train_scaler.fit_transform(train_ngram_features)
classifier = SGDClassifier(
    loss='log',
        random_state=0, 
        learning_rate='constant',
        eta0=0.02,
        max_iter=300, 
        early_stopping=True,
        validation_fraction=0.15
    )
classifier.fit(training_combined, train_labels)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
#test_scaler = StandardScaler(with_mean=False)
#test_scaled_features = test_scaler.fit_transform(test_ngram_features)
#test_predictions = lr.predict(test_combined)
test_predictions = classifier.predict(test_combined)

In [ ]:
test_report = classification_report(test_labels, test_predictions, output_dict=True)
# get metrics of interest
avg_rec = test_report['macro avg']['recall']*100
f1_neg = test_report['negative']['f1-score']
f1_pos = test_report['positive']['f1-score']
avg_f1 = np.mean([f1_neg, f1_pos])*100
acc = test_report['accuracy']*100
print(f'Macro avgRec: {avg_rec:.2f}%')
print(f'Macro avgF1: {avg_f1:.2f}%')
print(f'Acc: {acc:.2f}%')

lr = 'constant' -> 0.02, early stopping, valid_split = 15%
best ngrams = (1,2,3)

## Logistic regression
**Syntactic features**<br>
Macro avgRec: 39.994%<br>
Macro avgF1: 29.770%<br>
Acc: 44.554%

**Bag of ngrams (1,2,3)**<br>
Macro avgRec: 57.035%<br>
Macro avgF1: 52.657%<br>
Acc: 59.614%

**Lexicon**<br>
Macro avgRec: 39.40%<br>
Macro avgF1: 18.79%<br>
Acc: 46.93%

**Syntactic + bag of ngrams + lexicon**<br>
Macro avgRec: 60.64%<br>
Macro avgF1: 58.26%<br>
Acc: 60.09%

## SVM
**Syntactic features**<br>
Macro avgRec: 40.557%<br>
Macro avgF1: 31.716%<br>
Acc: 44.863%

**Bag of ngrams (1,2,3)**<br>
Macro avgRec: 56.571%<br>
Macro avgF1: 52.097%<br>
Acc: 59.175%

**Lexicon**
Macro avgRec: 37.353%<br>
Macro avgF1: 15.242%<br>
Acc: 46.475%

**Syntactic + bag of ngrams + lexicon**<br>
Macro avgRec: 60.329%<br>
Macro avgF1: 57.783%<br>
Acc: 58.540%<br>
lr = 'constant' -> 0.02, early stopping, valid_split = 15%